# fMRI Lab: Day 2 Validation

## EXPLORATORY ANALYSIS

Welcome back. Today you will find out whether the findings you presented in class hold up in an independent dataset.

---

## Part 1: Set Your Topic and Reload Data

Enter your topic below (the same one from Day 1), then run all the cells in order. Your Colab runtime is fresh, so you need to reinstall packages and re-download the data.

In [ ]:
# SET YOUR TOPIC (must match Day 1)
# Options: 'pain', 'depression', or 'anxiety'

TOPIC = 'depression'   # <-- change this to match your Day 1 assignment

In [ ]:
# Install packages and download data files
import subprocess, sys
subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'nilearn', 'statsmodels', '-q'])

import os, urllib.request
base_url = 'https://raw.githubusercontent.com/cmahlen/python-stats-demo/main/'
files_needed = [
    'lab_helpers.py', 'atlas_labels.txt', 'data/roi_mni_coords.npy',
    f'data/{TOPIC}_discovery.npz', f'data/{TOPIC}_validation.npz',
]
os.makedirs('data', exist_ok=True)
for f in files_needed:
    if not os.path.exists(f):
        urllib.request.urlretrieve(base_url + f, f)

import lab_helpers as helpers
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import pearsonr
import pandas as pd

print("Setup complete!")

In [ ]:
# Re-run discovery analysis to reconstruct your Day 1 results
helpers.load_dataset(TOPIC, 'discovery')
results = helpers.test_all_edges()

n_sig = (results['p'] < 0.05).sum()
print(f"Discovery dataset: {n_sig:,} edges significant at p < 0.05 (uncorrected)")
print(f"\nTop 5 findings from discovery:")
print(results.head(5)[['ROI_A', 'ROI_B', 'r', 'p']].to_string())

---

## Part 2: Commit Your Claimed Findings

**Before we load the validation data**, enter the specific edges you are planning to present. Copy the exact ROI names from your Day 1 notebook output.

This step matters: you are committing to your claims before seeing whether they replicate. That is what makes this a real test.

In [ ]:
# Enter the edges you are claiming as your findings.
# Copy the exact ROI names from your Day 1 results table.
# Format: ('ROI_A_name_here', 'ROI_B_name_here')

my_findings = [
    ('LH_DefaultB_PFCd_3', 'LH_TempPar_2'),   # <-- Replace with your actual edge(s)
    # Add more rows if you are claiming multiple findings:
    # ('ROI_A_name_here', 'ROI_B_name_here'),
]

# Enter the same analytic choices you used in Day 1:
covariates        = None   # e.g. ['Physical_Activity', 'Screen_Time'] or None
outlier_threshold = None   # e.g. 2 or None
subgroup          = None   # e.g. {'Sex': 0} or None

print("Your claimed findings:")
for i, (a, b) in enumerate(my_findings, 1):
    print(f"  {i}. {a} <-> {b}")
print(f"\nCovariates: {covariates}  |  Outlier threshold: {outlier_threshold}  |  Subgroup: {subgroup}")

In [ ]:
outcome_col = helpers.get_behavior().columns[0]

for roi_a, roi_b in my_findings:
    helpers.plot_edge(roi_a, roi_b, behavior_col=outcome_col,
                      covariates=covariates,
                      exclude_outliers=outlier_threshold,
                      subgroup=subgroup)

---

## Part 3: Validate Your Claimed Findings

Now load the validation dataset and test whether your claimed edges replicate. A finding replicates if it is statistically significant in the same direction in both datasets.

The code below is more complex than what you wrote on Day 1. You do not need to understand every line; just run the cell and focus on the output.

In [ ]:
print("=" * 80)
print("VALIDATION OF YOUR CLAIMED FINDINGS")
print("=" * 80)

claimed_results = []
for roi_a, roi_b in my_findings:
    # Discovery (using the same analytic choices you entered above)
    helpers.load_dataset(TOPIC, 'discovery')
    r_disc, p_disc, n_disc = helpers.test_edge(
        roi_a, roi_b, behavior_col=outcome_col,
        covariates=covariates, exclude_outliers=outlier_threshold, subgroup=subgroup
    )

    # Validation (same analytic choices applied to new data)
    helpers.load_dataset(TOPIC, 'validation')
    r_val, p_val, n_val = helpers.test_edge(
        roi_a, roi_b, behavior_col=outcome_col,
        covariates=covariates, exclude_outliers=outlier_threshold, subgroup=subgroup
    )

    same_direction = (r_val * r_disc) > 0
    if p_val < 0.05 and same_direction:
        replicated = 'YES'
    elif p_val < 0.05 and not same_direction:
        replicated = 'FLIPPED'
    else:
        replicated = 'NO'

    claimed_results.append({
        'ROI_A': roi_a,
        'ROI_B': roi_b,
        'Discovery_r': round(r_disc, 3),
        'Discovery_p': round(p_disc, 4),
        'Validation_r': round(r_val, 3),
        'Validation_p': round(p_val, 4),
        'Replicated': replicated,
    })

claimed_df = pd.DataFrame(claimed_results)
claimed_df.index = range(1, len(claimed_df) + 1)
print(claimed_df.to_string())

n_rep = (claimed_df['Replicated'] == 'YES').sum()
print(f"\n{n_rep} of {len(my_findings)} claimed finding(s) replicated.")

In [ ]:
# Scatter plots for your claimed findings (discovery then validation)
# Analytic choices from above are applied to both
for roi_a, roi_b in my_findings:
    print(f"\n{'='*60}")
    print(f"{roi_a} <-> {roi_b}")
    print(f"{'='*60}")

    print("Discovery set:")
    helpers.load_dataset(TOPIC, 'discovery')
    helpers.plot_edge(roi_a, roi_b, behavior_col=outcome_col,
                      covariates=covariates,
                      exclude_outliers=outlier_threshold,
                      subgroup=subgroup)

    print("Validation set:")
    helpers.load_dataset(TOPIC, 'validation')
    helpers.plot_edge(roi_a, roi_b, behavior_col=outcome_col,
                      covariates=covariates,
                      exclude_outliers=outlier_threshold,
                      subgroup=subgroup)

---

## Part 4: Validate Your Top 5 Discoveries

Now let's test the 5 statistically strongest edges from your discovery dataset, regardless of which ones you chose to present. This is an objective baseline.

After you see the results, compare: how many of your claimed findings overlap with the top 5? Which set replicated better, and why?

In [ ]:
top_5 = results.head(5)

print("=" * 80)
print("VALIDATION OF TOP 5 DISCOVERY EDGES (by p-value)")
print("=" * 80)

top5_results = []
for _, row in top_5.iterrows():
    r_disc = row['r']
    p_disc = row['p']

    helpers.load_dataset(TOPIC, 'validation')
    val_edge = helpers.get_edge(row['ROI_A'], row['ROI_B'])
    val_out = helpers.get_behavior()[outcome_col].values
    r_val, p_val = pearsonr(val_edge, val_out)

    same_direction = (r_val * r_disc) > 0
    if p_val < 0.05 and same_direction:
        replicated = 'YES'
    elif p_val < 0.05 and not same_direction:
        replicated = 'FLIPPED'
    else:
        replicated = 'NO'

    top5_results.append({
        'ROI_A': row['ROI_A'],
        'ROI_B': row['ROI_B'],
        'Discovery_r': round(r_disc, 3),
        'Discovery_p': round(p_disc, 4),
        'Validation_r': round(r_val, 3),
        'Validation_p': round(p_val, 4),
        'Replicated': replicated,
    })

top5_df = pd.DataFrame(top5_results)
top5_df.index = range(1, len(top5_df) + 1)
print(top5_df.to_string())

n_rep = (top5_df['Replicated'] == 'YES').sum()
print(f"\n{n_rep} of 5 top discoveries replicated.")

How many of your claimed findings (Part 3) overlap with the top 5? Which set replicated better, and why might that be?

---

## Part 5: Reflection Questions

Before we discuss as a group, take a moment to think about these questions:

1. **How confident were you that your FDR-corrected discoveries were real?**
   - What would have made you more or less confident?

2. **The multiple comparisons problem**
   - You narrowed from 23,220 edges to a specific network to get a gentler FDR correction. Is that the whole story?
   - How many different networks, covariates, and subgroups did you try before finding something that survived correction?
   - Does the FDR correction account for ALL the tests you actually ran?

3. **Analytic flexibility**
   - How many different analysis configurations did you try?
   - If you tried multiple approaches, which results did you choose to present and why?

4. **The extra credit incentive**
   - Did the extra credit incentive affect your analysis decisions?
   - Did you feel pressure to find results that survived FDR correction?
   - How is this similar to pressures in real academic research?

5. **Effect sizes**
   - How strong were the correlations you found (r values)?
   - Are these large enough to be practically meaningful?

Let's discuss these briefly before the class wrap-up.

---

## What Did We Learn?

If your top findings **did not replicate**, you are in good company. This is the norm for underpowered, exploratory analyses. The findings were real in the discovery dataset, but they were likely false positives driven by chance.

This is not a failure. It is the lesson. The reproducibility crisis in science is largely caused by exactly what you just did: testing many things, finding something that works, and reporting it as if it were the only thing you tested.

**The hypothesis-driven group** pre-registered their analysis before running it. We will find out whether their approach fared better and why.

If your findings **did replicate**: think carefully about whether you ran a truly hypothesis-driven analysis, or whether you found your "hypothesis" after exploring the data.